In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import mahotas
import h5py
import imageio as iio
import os
from os import listdir
from itertools import chain
import pickle as pkl


In [2]:
def read_images(string):
    AllImages = []
    hue_features_all = []
    haralick_features_all = []
    for i in range(0,10):
        dirname = '../datasets/'+string+'/'+string+'/n'+str(i)
        images = []
        for image in os.listdir(dirname):
            img = iio.imread(dirname+'/' +image)

            image_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            hue_features = cv2.HuMoments(cv2.moments(image_gray)).flatten()

            haralick_features = mahotas.features.haralick(image_gray).mean(axis=0)

            hue_features_all.append(hue_features)
            haralick_features_all.append(haralick_features)
            images.append(img)
        AllImages.append(images)
    return AllImages ,hue_features_all ,haralick_features_all


In [3]:
def histcaclulate(AllImages , rgb):   
    AllHists = []
    for images in AllImages:
        hists = []
        for image in images:
            hist = cv2.calcHist(image,[rgb],None,[256],[0,256]) 
            hists.append(hist)
        AllHists.append(hists)
    return AllHists

In [4]:
def concatenatehists(AllHists0,AllHists1,AllHists2):   
    Allhists = []
    for i in range(10):
        folderhists = []
        for j in range(len(AllHists0[i])):
            imagehist = np.concatenate([AllHists0[i][j] , AllHists1[i][j] , AllHists2[i][j]]) 
            folderhists.append(imagehist)
        Allhists.append(folderhists)
    return Allhists


In [5]:
def flator(lists):
    result = []
    for folder in lists:
        images = []
        for image in folder:
            images.append(image.ravel())
        result.append(images)
    return result
        

In [6]:
def findlabels(Allhists):
    lables = []
    for i in range(len(Allhists)):
        for j in range(len(Allhists[i])):
            lables.append(i)
    return lables


In [8]:
def handle(string):
    All = read_images(string)
    AllImages = All[0]
    hue_features_all = All[1]
    haralick_features_all = All[2]
    AllHists0 = histcaclulate(AllImages , 0)
    AllHists1 = histcaclulate(AllImages , 1)
    AllHists2 = histcaclulate(AllImages , 2)
    del All
    del AllImages
    Allhists = concatenatehists(AllHists0,AllHists1,AllHists2)   
    del AllHists0
    del AllHists1
    del AllHists2
    flatlist = flator(Allhists)
    lables = findlabels(Allhists)
    flatlist = list(chain.from_iterable(flatlist))
    all_featurs = np.concatenate((flatlist , hue_features_all, haralick_features_all), axis = 1)
    del flatlist
    return all_featurs , lables

In [9]:
X_train ,y_train = handle('training')

In [ ]:
with open('X_train.pkl', 'wb') as f:
    pickle.dump(X_train, f)
with open('X_train.pkl', 'wb') as f:
    pickle.dump(X_train, f)

In [10]:
X_test ,y_test = handle('validation')

In [79]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(iterations=1000 ,learning_rate=0.5,depth = 5 ,eval_metric='Accuracy')

In [80]:
model.fit(all_featurs,lables)

0:	learn: 0.2550091	total: 1.54s	remaining: 25m 40s
1:	learn: 0.2950820	total: 2.37s	remaining: 19m 41s
2:	learn: 0.3251366	total: 2.73s	remaining: 15m 9s
3:	learn: 0.3561020	total: 3.1s	remaining: 12m 52s
4:	learn: 0.3779599	total: 3.53s	remaining: 11m 41s
5:	learn: 0.3916211	total: 3.94s	remaining: 10m 53s
6:	learn: 0.4134791	total: 4.35s	remaining: 10m 17s
7:	learn: 0.4326047	total: 4.82s	remaining: 9m 57s
8:	learn: 0.4298725	total: 5.25s	remaining: 9m 37s
9:	learn: 0.4316940	total: 5.63s	remaining: 9m 17s
10:	learn: 0.4635701	total: 5.99s	remaining: 8m 58s
11:	learn: 0.4826958	total: 6.35s	remaining: 8m 42s
12:	learn: 0.5173042	total: 6.71s	remaining: 8m 29s
13:	learn: 0.5163934	total: 7.07s	remaining: 8m 18s
14:	learn: 0.5409836	total: 7.43s	remaining: 8m 7s
15:	learn: 0.5382514	total: 7.78s	remaining: 7m 58s
16:	learn: 0.5582878	total: 8.14s	remaining: 7m 50s
17:	learn: 0.5601093	total: 8.48s	remaining: 7m 42s
18:	learn: 0.5692168	total: 8.84s	remaining: 7m 36s
19:	learn: 0.57923

158:	learn: 0.9963570	total: 1m 12s	remaining: 6m 23s
159:	learn: 0.9972678	total: 1m 12s	remaining: 6m 22s
160:	learn: 0.9981785	total: 1m 13s	remaining: 6m 21s
161:	learn: 0.9981785	total: 1m 13s	remaining: 6m 20s
162:	learn: 0.9972678	total: 1m 13s	remaining: 6m 19s
163:	learn: 0.9981785	total: 1m 14s	remaining: 6m 18s
164:	learn: 0.9990893	total: 1m 14s	remaining: 6m 18s
165:	learn: 0.9990893	total: 1m 15s	remaining: 6m 18s
166:	learn: 0.9990893	total: 1m 15s	remaining: 6m 18s
167:	learn: 0.9990893	total: 1m 16s	remaining: 6m 17s
168:	learn: 0.9990893	total: 1m 16s	remaining: 6m 16s
169:	learn: 0.9990893	total: 1m 17s	remaining: 6m 16s
170:	learn: 0.9990893	total: 1m 17s	remaining: 6m 15s
171:	learn: 0.9990893	total: 1m 17s	remaining: 6m 15s
172:	learn: 0.9990893	total: 1m 18s	remaining: 6m 15s
173:	learn: 0.9990893	total: 1m 19s	remaining: 6m 16s
174:	learn: 1.0000000	total: 1m 19s	remaining: 6m 16s
175:	learn: 0.9990893	total: 1m 20s	remaining: 6m 16s
176:	learn: 1.0000000	total:

KeyboardInterrupt: 